In [1]:
!pip3 install torch torchvision torchaudio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
!ls gdrive/My\ Drive/data

 data	     data_alfonso   full_data.zip   task1   task3   task5   task7
'data (1)'   data.zip	    old_data.zip    task2   task4   task6


In [ ]:
!unzip gdrive/My\ Drive/data/data.zip > /dev/null

# THESE BOTTOM COMMAND IS FOR ALFONSO SINCE I HAD TO CREATE A SHORTCUT TO THE OG DATASET

In [3]:
!ls gdrive/My\ Drive\/data_alfonso/data

 data	        data_all_final.zip   full_data.zip   task2   task5
'data (1)'      data_smaller.zip     old_data.zip    task3   task6
 data_alfonso   data.zip	     task1	     task4   task7


In [4]:
!unzip  gdrive/My\ Drive\/data_alfonso/data/data_all_final.zip > /dev/nully

In [ ]:
!pip install wandb -qqq
import wandb
wandb.login()

wandb.init(project="jupyter-proj",
           config={
               "batch_size": 100,
               "learning_rate": 0.01,
           })

     |████████████████████████████████| 1.9 MB 27.6 MB/s 
     |████████████████████████████████| 182 kB 72.8 MB/s 
     |████████████████████████████████| 168 kB 56.3 MB/s 
     |████████████████████████████████| 62 kB 1.4 MB/s 
     |████████████████████████████████| 168 kB 80.1 MB/s 
     |████████████████████████████████| 166 kB 54.8 MB/s 
     |████████████████████████████████| 166 kB 81.0 MB/s 
     |████████████████████████████████| 162 kB 81.0 MB/s 
     |████████████████████████████████| 162 kB 68.7 MB/s 
     |████████████████████████████████| 158 kB 83.1 MB/s 
     |████████████████████████████████| 157 kB 76.9 MB/s 
     |████████████████████████████████| 157 kB 82.7 MB/s 
     |████████████████████████████████| 157 kB 83.6 MB/s 
     |████████████████████████████████| 157 kB 64.9 MB/s 
     |████████████████████████████████| 157 kB 79.8 MB/s 
     |████████████████████████████████| 157 kB 75.7 MB/s 
     |████████████████████████████████| 157 kB 79.4 MB/s 
     |██████████

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: alv2145 (dl4cv-project). Use `wandb login --relogin` to force relogin


In [ ]:
# Unzip Dataset
"""import zipfile

with zipfile.ZipFile("./data.zip","r") as z:
    z.extractall(".")
    """

In [5]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, random_split 
import torch.nn.functional as F
import torchvision
from torchvision import transforms, datasets, models
from torchvision.datasets import ImageFolder 
from PIL import Image
import matplotlib.pyplot as plt
import os
import glob 
import random
import shutil

In [6]:
# move in all of our collected data
task1_dataset = glob.glob(os.path.join('./data/task1/', '*.png'))
task2_dataset = glob.glob(os.path.join('./data/task2/', '*.png'))
task3_dataset = glob.glob(os.path.join('./data/task3/', '*.png'))
task4_dataset = glob.glob(os.path.join('./data/task4/', '*.png'))
task5_dataset = glob.glob(os.path.join('./data/task5/', '*.png'))
task6_dataset = glob.glob(os.path.join('./data/task6/', '*.png'))
task7_dataset = glob.glob(os.path.join('./data/task7/', '*.png'))


if not os.path.exists("./training"):
  os.mkdir("./training")
  os.mkdir("./training/before")
  os.mkdir("./training/after")
if not os.path.exists("./validation"):
  os.mkdir("./validation")
  os.mkdir("./validation/before")
  os.mkdir("./validation/after")
if not os.path.exists("./test"): 
  os.mkdir("./test")
  os.mkdir("./test/before")
  os.mkdir("./test/after")

# this needs to be changed to 7 when we download data correctly
tasks = [task1_dataset, task2_dataset, task3_dataset, task4_dataset, task5_dataset, task6_dataset, task7_dataset]
# now for each task randomly generate numbers for before and after to split
for i in range(7):
  current_task = tasks[i]
  training_size = int(len(current_task) * .8)
  test_size = int(len(current_task) * .1)
  val_size = int(len(current_task) * .1)

  # randomly choose an element
  generated_list = [j+1 for j in range(int(len(current_task) / 2))] 
  
  # add elements to our training array
  for k in range(int(training_size / 2)):
    random_index = random.randint(0,len(generated_list)-1)
    random_element = generated_list[random_index]

    # add the after and before images corresponding to this number 
    shutil.copy(f"./data/task{i+1}/before{random_element}.png", f"./training/before/task{i+1}_before_{random_element}.png")
    shutil.copy(f"./data/task{i+1}/after{random_element}.png", f"./training/after/task{i+1}_after_{random_element}.png")

    # remove that this element can be selected again
    del generated_list[random_index]


  # add elements to our test
  for k in range(int(test_size / 2)):

    random_index = random.randint(0,len(generated_list)-1)
    random_element = generated_list[random_index]

    # add the after and before images corresponding to this number 
    shutil.copy(f"./data/task{i+1}/before{random_element}.png", f"./test/before/task{i+1}_before_{random_element}.png")
    shutil.copy(f"./data/task{i+1}/after{random_element}.png", f"./test/after/task{i+1}_after_{random_element}.png")

    # remove that this element can be selected again
    del generated_list[random_index]

  
  # add elements to our validation
  for k in range(int(val_size / 2)):

    random_index = random.randint(0,len(generated_list)-1)
    random_element = generated_list[random_index]

    # add the after and before images corresponding to this number 
    shutil.copy(f"./data/task{i+1}/before{random_element}.png", f"./validation/before/task{i+1}_before_{random_element}.png")
    shutil.copy(f"./data/task{i+1}/after{random_element}.png", f"./validation/after/task{i+1}_after_{random_element}.png")

    # remove that this element can be selected again
    del generated_list[random_index]

  


In [7]:
# Image transforms
dim = 512
def get_train_transform():
    return transforms.Compose([
    transforms.Resize((dim, dim)),
    transforms.ToTensor(),
    transforms.Normalize((0, 0, 0),(1, 1, 1))
    ])

def get_val_transform(): 
    return transforms.Compose([
    transforms.Resize((dim,dim)),
    transforms.ToTensor(),
    transforms.Normalize((0, 0, 0),(1, 1, 1))
    ])

In [8]:
# PyTorch Dataset
# Source: https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset
class CreateDataset(Dataset):
    
    def __init__(self, imgs, class_to_int, folder_type, transforms = None):
        super().__init__()
        self.imgs = imgs
        self.class_to_int = class_to_int
        self.transforms = transforms
        self.folder_type = folder_type
        
    def __getitem__(self, idx):
        image_name = self.imgs[idx]
        #print(image_name + "\n")
        indicators = image_name.split("_")
        task_num = indicators[0]
        image_num = indicators[2]
        
        before_dir = f"./{self.folder_type}/before/"
        before_img = self.transforms(Image.open(before_dir + image_name).convert('RGB'))
        after_dir = f"./{self.folder_type}/after/"
        after_image_name = task_num + "_after_" + image_num
        after_img = self.transforms(Image.open(after_dir + after_image_name).convert('RGB'))

        label = self.class_to_int[task_num]
        label = torch.tensor(label, dtype = torch.float32)

        image_list = [before_img, after_img]

        return image_list, label
            
        
    def __len__(self):
        return len(self.imgs)

In [9]:
train_dir = "./training/before/"
train_images = os.listdir(train_dir) 

val_dir = "./validation/before/"
val_images = os.listdir(val_dir) 

test_dir = "./test/before/"
test_images = os.listdir(test_dir) 

In [10]:
# Create Datasets to be used by Data Loaders
class_mapping = {"task1" : 0, "task2" : 1, "task3" : 2, "task4": 3, "task5": 4, "task6": 5, "task7": 6}
train_data = CreateDataset(train_images, class_mapping, folder_type = "training", transforms = get_train_transform())
val_data = CreateDataset(val_images, class_mapping, folder_type = "validation", transforms = get_val_transform())
test_data = CreateDataset(test_images, class_mapping, folder_type = "test", transforms = get_val_transform())

In [113]:
# Data Loaders
batch_size = 16
num_workers = 1
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=num_workers)
validation_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [12]:
print('Num training images: ' + str(len(train_loader.dataset)))
print('Num validation images: ' + str(len(validation_loader.dataset)))
print('Num test images: ' + str(len(test_loader.dataset)))

Num training images: 347
Num validation images: 40
Num test images: 40


In [13]:
from torchvision.models import resnet18, ResNet18_Weights


ResNet18 = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
ResNet18_mask = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
ResNet18

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [14]:
# CUT OFF THE LAYERS AT THE END

ResNet18.fc = nn.Sequential(
   nn.Linear(512, 512)
)

ResNet18_mask.fc = nn.Sequential(
   nn.Linear(512, 512)
)




In [ ]:
ResNet18

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [15]:
# COMBINE BOTH MODELS INTO ONE CLASS
class ActionModel(nn.Module):
  def __init__(self):
    super(ActionModel, self).__init__()

    # NETWORK A IS OUR FIRST RESNET that encodes the images 
    self.img_encoder = ResNet18
    self.mask_encoder = ResNet18_mask

    self.fc = nn.Sequential(
        nn.Linear(1536,1024),
        nn.ReLU(),
        nn.Linear(1024, 512),
        nn.ReLU(),
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Linear(256, 7),
        nn.Softmax()
    )


  # forward propogation into our model
  def forward(self, before_image, after_image, mask_image):

      # run forward prop on the before image
      before_encoded = self.img_encoder(before_image)

      # run forward prop on the after image
      after_encoded = self.img_encoder(after_image)

      mask_encoded = self.mask_encoder(mask_image)

      # combine our encodings
      images_combined = torch.cat((before_encoded, after_encoded, mask_encoded), 1)

      

      #run final layer of model and return the highest value
      return self.fc(images_combined)


In [125]:
#Define Utility Functions 
import numpy as np
#Accuracy function 
def accuracy(preds, labels):
  _, pred = torch.max(preds, dim=1)
  return torch.sum(pred==labels).item()/len(labels)

def check_model_accuracy(model, dloader):
    model.eval()
    acc = 0
    mistakes = []
    incorrect_before = []
    incorrect_after = []
    incorrect_mask = []
    incorrect_labels = []
    correct_labels = []

    correct_before = []
    correct_after = []
    correct_mask = []
    correct_labels_for_correct = []

    for batch, (X, y) in enumerate(dloader):
        #convert batch into variable that can have gradients applied to them
        X[0] = X[0].to(device)
        X[1] = X[1].to(device)

        masks = torch.unsqueeze(torch.all(torch.eq(X[0], X[1]),axis=1), axis = 1)

        masks_numbers = np.empty(masks.shape)
        masks_numbers[np.where(masks.cpu() == True)] = 255
        masks_numbers[np.where(masks.cpu() == False)] = 0

        masks_numbers_tensor = torch.from_numpy(masks_numbers.astype(np.double)).float()

        rgb_masks = masks_numbers_tensor.repeat(1,3,1,1)
        rgb_masks = rgb_masks.to(device)
        y = y.to(device)
        #forward pass thru net
        preds = model(X[0], X[1], rgb_masks)

        prediction_final = torch.argmax(preds, dim=1).float()
        prediction_numpy = prediction_final.cpu().detach().numpy() 
        target_numpy = y.cpu().numpy()


        for i in range(len(prediction_numpy)):
          prediction = int(prediction_numpy[i])
          target = int(target_numpy[i])

          if prediction != target:
       
            incorrect_before.append(X[0][i].clamp(0,1).cpu().numpy())
            incorrect_after.append(X[1][i].clamp(0,1).cpu().numpy())
            incorrect_mask.append(rgb_masks[i].clamp(0,1).cpu().numpy() )
            incorrect_labels.append(prediction)
            correct_labels.append(target)

          else:
            correct_before.append(X[0][i].clamp(0,1).cpu().numpy())
            correct_after.append(X[1][i].clamp(0,1).cpu().numpy())
            correct_mask.append(rgb_masks[i].clamp(0,1).cpu().numpy())
            correct_labels_for_correct.append(target)


        #check accuracy
        acc+=accuracy(preds, y)

    return acc/len(dloader), correct_before, correct_after, correct_mask, correct_labels_for_correct, incorrect_before, incorrect_after, incorrect_mask, incorrect_labels, correct_labels

In [ ]:
model = ActionModel()

In [ ]:
model

ActionModel(
  (img_encoder): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, t

In [ ]:
# TRAIN THE MODEL
# %%wandb

import numpy as np
from torchvision import transforms

#Freeze initial layers

for i, child in enumerate(model.img_encoder.children()):
    if i<9:
        for param in child.parameters():
            param.requires_grad = False 
    else:
        for param in child.parameters():
            param.requires_grad = True

#init SGD optimizer & CrossEntropyLoss function
optim = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=.9 )
loss_fn = nn.CrossEntropyLoss()

#put on device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


n_epochs = 20
step=0

#Loop thru epochs 
for i in range(n_epochs):
    val_losses = []
    
    #loop thru samples
    for idx, (X, y) in enumerate(train_loader):
        
        X[0] = X[0].to(device)
        X[1] = X[1].to(device)
        masks = torch.unsqueeze(torch.all(torch.eq(X[0], X[1]),axis=1), axis = 1)

        masks_numbers = np.empty(masks.shape)
        masks_numbers[np.where(masks.cpu() == True)] = 255
        masks_numbers[np.where(masks.cpu() == False)] = 0

        masks_numbers_tensor = torch.from_numpy(masks_numbers.astype(np.double)).float()

        rgb_masks = masks_numbers_tensor.repeat(1,3,1,1)
        # print(X[0].shape)
        # print(X[1].shape)
        # print(mask.shape)

        mask = masks.repeat(1,3,1,1)
        # print(mask.shape)

        # print(X[0][0].shape)
        # print(X[1][0].shape)
        # print(mask[0].shape)

        result = np.concatenate((X[0][0].cpu().numpy(), X[1][0].cpu().numpy(), mask[0].cpu().numpy()), axis=2)

        y = y.to(device)
        
        #make predictions
        rgb_masks = rgb_masks.to(device)
        preds = model(X[0], X[1], rgb_masks)

        p_idx = np.argmax(preds.cpu().detach().numpy()[0], axis=0)
        gt = str(int(y.cpu().detach()[0].item()))

        #calculate loss 
        loss = loss_fn(preds.cpu(), y.cpu().to(torch.long))
        print(loss)

        #log the training loss 
        # wandb.log({"train_loss": loss})

        #backprop
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        #logging 
        if step%50==0:

            #visualize a result from the training
            plt.imsave("./training_images/epoch_"+str(i)+"_step_"+str(idx)+"_GT_"+gt+"_P_"+str(p_idx)+".jpg", np.moveaxis(result,0,-1))

            #calculate accuracy 
            model.eval()
            val_acc=0
            val_loss=0

            with torch.no_grad():
                for batch, (X, y) in enumerate(validation_loader):

                    X[0] = X[0].to(device)
                    X[1] = X[1].to(device)
          
                    masks = torch.unsqueeze(torch.all(torch.eq(X[0], X[1]),axis=1), axis = 1)

                    masks_numbers = np.empty(masks.shape)
                    masks_numbers[np.where(masks.cpu() == True)] = 255
                    masks_numbers[np.where(masks.cpu() == False)] = 0

                    masks_numbers_tensor = torch.from_numpy(masks_numbers.astype(np.double)).float()

                    rgb_masks = masks_numbers_tensor.repeat(1,3,1,1)
                    rgb_masks = rgb_masks.to(device)
                    y = y.to(device)
                    preds = model(X[0],X[1], rgb_masks)
                    
                    val_loss+=loss_fn(preds.cpu(), y.cpu().to(torch.long))
                    val_acc+=accuracy(preds.cpu(), y.cpu())
                
                #normalize val accuracy and loss
                val_acc = val_acc / len(validation_loader)
                val_loss = val_loss / len(validation_loader)

                #wandb.log({"val_loss": val_loss})
                #wandb.log({"val_acc": val_acc})
                    
                #early stopping
                if len(val_losses)>5:
                    if val_loss >= (sum(val_losses[i-5:i])/5):
                        print("Stopped at Epoch: ", idx)
                        break

                val_losses.append(val_loss)
        step+=1

    if len(val_losses) > 0:
      print("Epoch: ", i, " Val Loss: ", val_losses[-1], " Val Accuracy: ", val_acc)

tensor(1.9461, grad_fn=<NllLossBackward0>)
tensor(1.9442, grad_fn=<NllLossBackward0>)
tensor(1.9472, grad_fn=<NllLossBackward0>)
tensor(1.9485, grad_fn=<NllLossBackward0>)
tensor(1.9481, grad_fn=<NllLossBackward0>)
tensor(1.9430, grad_fn=<NllLossBackward0>)
tensor(1.9501, grad_fn=<NllLossBackward0>)
tensor(1.9428, grad_fn=<NllLossBackward0>)
tensor(1.9474, grad_fn=<NllLossBackward0>)
tensor(1.9451, grad_fn=<NllLossBackward0>)
tensor(1.9471, grad_fn=<NllLossBackward0>)
tensor(1.9492, grad_fn=<NllLossBackward0>)
tensor(1.9445, grad_fn=<NllLossBackward0>)
tensor(1.9494, grad_fn=<NllLossBackward0>)
tensor(1.9470, grad_fn=<NllLossBackward0>)
tensor(1.9427, grad_fn=<NllLossBackward0>)
tensor(1.9433, grad_fn=<NllLossBackward0>)
tensor(1.9467, grad_fn=<NllLossBackward0>)
tensor(1.9453, grad_fn=<NllLossBackward0>)
tensor(1.9439, grad_fn=<NllLossBackward0>)
tensor(1.9439, grad_fn=<NllLossBackward0>)
tensor(1.9424, grad_fn=<NllLossBackward0>)
Epoch:  0  Val Loss:  tensor(1.9464)  Val Accuracy:  0

In [ ]:
# TEST THE MODEL

#Evaluate the accuracy on the test set.
check_model_accuracy(model, test_loader)

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/container.py:139: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


0.16666666666666666

In [ ]:
#Unfreeze all layers.

for i, param in enumerate(model.parameters()):
    param.requires_grad = True 

#Continue Fine Tuning

#init SGD optimizer & CrossEntropyLoss function
optim = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=.9 )
loss_fn = nn.CrossEntropyLoss()

#put on device
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


n_epochs = 20
step=0

#Loop thru epochs 
for i in range(n_epochs):
    val_losses = []
    
    #loop thru samples
    for idx, (X, y) in enumerate(train_loader):
        
        X[0] = X[0].to(device)
        X[1] = X[1].to(device)
        masks = torch.unsqueeze(torch.all(torch.eq(X[0], X[1]),axis=1), axis = 1)

        masks_numbers = np.empty(masks.shape)
        masks_numbers[np.where(masks.cpu() == True)] = 255
        masks_numbers[np.where(masks.cpu() == False)] = 0

        masks_numbers_tensor = torch.from_numpy(masks_numbers.astype(np.double)).float()

        rgb_masks = masks_numbers_tensor.repeat(1,3,1,1)
        # print(X[0].shape)
        # print(X[1].shape)
        # print(mask.shape)

        mask = masks.repeat(1,3,1,1)
        # print(mask.shape)

        # print(X[0][0].shape)
        # print(X[1][0].shape)
        # print(mask[0].shape)

        result = np.concatenate((X[0][0].cpu().numpy(), X[1][0].cpu().numpy(), mask[0].cpu().numpy()), axis=2)

        y = y.to(device)
        
        #make predictions
        rgb_masks = rgb_masks.to(device)
        preds = model(X[0], X[1], rgb_masks)

        p_idx = np.argmax(preds.cpu().detach().numpy()[0], axis=0)
        gt = str(int(y.cpu().detach()[0].item()))

        #calculate loss 
        loss = loss_fn(preds.cpu(), y.cpu().to(torch.long))
        print(loss)

        #log the training loss 
        # wandb.log({"train_loss": loss})

        #backprop
        optim.zero_grad()
        loss.backward()
        optim.step()
        
        #logging 
        if step%50==0:

            #visualize a result from the training
            plt.imsave("./training_images/epoch_"+str(i)+"_step_"+str(idx)+"_GT_"+gt+"_P_"+str(p_idx)+".jpg", np.moveaxis(result,0,-1))

            #calculate accuracy 
            model.eval()
            val_acc=0
            val_loss=0

            with torch.no_grad():
                for batch, (X, y) in enumerate(validation_loader):

                    X[0] = X[0].to(device)
                    X[1] = X[1].to(device)
          
                    masks = torch.unsqueeze(torch.all(torch.eq(X[0], X[1]),axis=1), axis = 1)

                    masks_numbers = np.empty(masks.shape)
                    masks_numbers[np.where(masks.cpu() == True)] = 255
                    masks_numbers[np.where(masks.cpu() == False)] = 0

                    masks_numbers_tensor = torch.from_numpy(masks_numbers.astype(np.double)).float()

                    rgb_masks = masks_numbers_tensor.repeat(1,3,1,1)
                    rgb_masks = rgb_masks.to(device)
                    y = y.to(device)
                    preds = model(X[0],X[1], rgb_masks)
                    
                    val_loss+=loss_fn(preds.cpu(), y.cpu().to(torch.long))
                    val_acc+=accuracy(preds.cpu(), y.cpu())
                
                #normalize val accuracy and loss
                val_acc = val_acc / len(validation_loader)
                val_loss = val_loss / len(validation_loader)

                #wandb.log({"val_loss": val_loss})
                #wandb.log({"val_acc": val_acc})
                    
                #early stopping
                if len(val_losses)>5:
                    if val_loss >= (sum(val_losses[i-5:i])/5):
                        print("Stopped at Epoch: ", idx)
                        break

                val_losses.append(val_loss)
        step+=1

    if len(val_losses) > 0:
      print("Epoch: ", i, " Val Loss: ", val_losses[-1], " Val Accuracy: ", val_acc)

tensor(1.8726, grad_fn=<NllLossBackward0>)
tensor(1.9025, grad_fn=<NllLossBackward0>)
tensor(1.8597, grad_fn=<NllLossBackward0>)
tensor(1.9497, grad_fn=<NllLossBackward0>)
tensor(1.9016, grad_fn=<NllLossBackward0>)
tensor(2.0902, grad_fn=<NllLossBackward0>)
tensor(1.9339, grad_fn=<NllLossBackward0>)
tensor(2.0163, grad_fn=<NllLossBackward0>)
tensor(1.8367, grad_fn=<NllLossBackward0>)
tensor(1.9872, grad_fn=<NllLossBackward0>)
tensor(1.9158, grad_fn=<NllLossBackward0>)
tensor(1.9233, grad_fn=<NllLossBackward0>)
tensor(1.9010, grad_fn=<NllLossBackward0>)
tensor(1.7114, grad_fn=<NllLossBackward0>)
tensor(2.0731, grad_fn=<NllLossBackward0>)
tensor(1.9517, grad_fn=<NllLossBackward0>)
tensor(1.7678, grad_fn=<NllLossBackward0>)
tensor(2.0420, grad_fn=<NllLossBackward0>)
tensor(1.9551, grad_fn=<NllLossBackward0>)
tensor(1.8757, grad_fn=<NllLossBackward0>)
tensor(1.7710, grad_fn=<NllLossBackward0>)
tensor(1.7396, grad_fn=<NllLossBackward0>)
Epoch:  0  Val Loss:  tensor(1.9228)  Val Accuracy:  0

In [ ]:
# TEST THE MODEL

#Evaluate the accuracy on the test set.
accuracy, mistakes, incorrect_labels, correct_labels = check_model_accuracy(model, test_loader)

RuntimeError: ignored

In [ ]:
torch.save(model.state_dict(), "./model_learning_rate_1e-3.pth")

In [20]:
newModel = ActionModel()
newModel.load_state_dict(torch.load("./model_learning_rate_1e-3.pth"))


<All keys matched successfully>

In [123]:
import gc
torch.cuda.empty_cache()
gc.collect()

23

In [127]:
#put on device
device = "cuda" if torch.cuda.is_available() else "cpu"
newModel.to(device)

accuracy , correct_before, correct_after, correct_mask, correct_labels_for_correct, incorrect_before, incorrect_after, incorrect_mask, incorrect_labels, correct_labels = check_model_accuracy(newModel, test_loader)

In [128]:
print(accuracy)

0.5


In [129]:

# let us plot the first 5 sample mistakes
for i in range(5):

  mistake_img_before = incorrect_before[i]
  mistake_img_after = incorrect_after[i]
  mistake_img_mask =  incorrect_mask[i]


  result = np.concatenate((mistake_img_before, mistake_img_after, mistake_img_mask), axis=2)
  result = np.moveaxis(result, 0, -1)

  plt.imsave("./sample_mistakes/predicted_" +  str(incorrect_labels[i]) + "_correct_"+ str(correct_labels[i]) + ".jpg", result)
  # plt.show()
  print("PREDICTED LABEL: " + str(incorrect_labels[i]))
  print("CORRECT LABEL: " + str(correct_labels[i]))

PREDICTED LABEL: 0
CORRECT LABEL: 1
PREDICTED LABEL: 4
CORRECT LABEL: 2
PREDICTED LABEL: 0
CORRECT LABEL: 5
PREDICTED LABEL: 0
CORRECT LABEL: 2
PREDICTED LABEL: 0
CORRECT LABEL: 3


In [130]:

# let us plot the first 5 sample mistakes

for i in range(5):

  correct_img_before = correct_before[i]
  correct_img_after = correct_after[i]
  correct_img_mask =  correct_mask[i]

  result = np.concatenate((correct_img_before, correct_img_after, correct_img_mask), axis=2)
  result = np.moveaxis(result, 0, -1)
  print(result.shape)
  plt.imsave("./sample_correct/correct_"+ str(correct_labels_for_correct[i]) + ".jpg", result)


(512, 1536, 3)
(512, 1536, 3)
(512, 1536, 3)
(512, 1536, 3)
(512, 1536, 3)
